# Decision Tree

* a non-parametric supervised learning approach used for classification and regression application. It splits the data based on feature values to create branches and leaf nodes, aiming to separate classes as cleanly as possible.
* Types of DT: 
    1. ID3 - measures data impurity at a node using entropy and selects the feature that provides the highest information gain
    2. CART - uses Gini impurity to decide how to split the data
* Entropy based splitting: The Information Gain (IG) measures how much entropy decreases after the split.

<div style="text-align: center;">
    <img src="/Users/payal/Desktop/Generalized-ML/Core ML/Decision_tree/formula.png" alt="Decision Tree Formula" width="600">
</div>

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter


In [4]:
data = load_iris()
x = data.data[:,[2,3]] # petal length and petal width
y = data.target


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [10]:
def gini_impurity(y):
    classes, counts = np.unique(y, return_counts=True)
    prob = counts / counts.sum()
    return 1 - np.sum(prob ** 2)


In [11]:
def split_data(x,y, feature_id, threshold):
    left_id = x[:, feature_id] <= threshold
    right_id = x[:, feature_id] > threshold
    return x[left_id], y[left_id], x[right_id], y[right_id]

In [12]:
class Node:
    def __init__(self, feature=None, left=None, right=None, threshold= None, value=None):
        self.feature = feature
        self.left = left
        self.right = right
        self.threshold = threshold
        self.value = value

In [13]:
## Function to find best split point
def best_split(x, y):
    best_feature, best_thresh, best_gini = None, None, 1
    n_features = x.shape[1]

    for feature_id in range(n_features):
        thresholds = np.unique(x[:, feature_id])
        for thresh in thresholds:
            x_left, y_left, x_right, y_right = split_data(x, y, feature_id, thresh)
            if len(y_left)==0 or len(y_right)==0:
                continue

            gini_split = (len(y_left)/len(y)) * gini_impurity(y_left) + (len(y_right)/len(y)) * gini_impurity(y_right)

            if gini_split < best_gini:
                best_gini = gini_split
                best_feature = feature_id
                best_thresh = thresh

    return best_feature, best_thresh

In [18]:
def build_decision_tree(x,y,depth=0,max_depth=3):

    if len(set(y)) == 1 or depth>= max_depth:
        value = Counter(y).most_common(1)[0][0]
        return Node(value=value)
    
    feature, thresh = best_split(x,y)

    if feature is None:
        value = Counter(y).most_common(1)[0][0]
        return Node(value=value)
    
    x_left, y_left, x_right, y_right = split_data(x,y, feature, thresh)

    left_node = build_decision_tree(x_left, y_left, depth+1, max_depth)
    right_node = build_decision_tree(x_right, y_right, depth+1, max_depth)

    return Node(feature=feature, threshold=thresh, left=left_node, right=right_node)

In [19]:
tree = build_decision_tree(x_train, y_train, max_depth=3)


In [20]:
def predict_one(node, x):
    if node.value is not None:
        return node.value
    
    if x[node.feature] <= node.threshold:
        return predict_one(node.left, x)
    else:
        return predict_one(node.right, x)
    
def predict(tree, X):
    return np.array([predict_one(tree, x) for x in X])


In [21]:
y_pred = predict(tree, x_test)

In [22]:
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.967
